<a href="https://colab.research.google.com/github/OUCTheoryGroup/pytorch_demo/blob/master/01_Using_CNN_for_dogs_vs_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##使用VGG模型进行猫狗大战

原文见：https://github.com/mlelarge/dataflowr/blob/master/CEA_EDF_INRIA/01_intro_DLDIY_colab.ipynb

结合自己的理解，有一定的修改。在本教程中，我们将建一个模型来完成 Kaggle 中的猫狗大战竞赛题目。在这个比赛中，有25000张标记好的猫和狗的图片用做训练，有12500张图片用做测试。这个竞赛是2013年开展的，如果你能够达到80%的准确率，在当年是一个 state-of-the-art 的成绩。

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import time
import json


# 判断是否存在GPU设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

## 1. 下载数据

Jeremy Howard 提供了数据的下载，链接为：http://files.fast.ai/data/dogscats.zip 

在他整理的数据集中，猫和狗的图片放在单独的文件夹中， 同时还提供了一个Validation数据。如果没有GPU设备，请减少用做训练的图像数据量即可。

因为这个代码需要在colab上跑，速度会相对较慢。因此，我们重新整理了数据，制作了新的数据集，训练集包含1800张图(猫的图片900张，狗的图片900张)，测试集包含2000张图。下载地址为：http://fenggao-image.stor.sinaapp.com/dogscats.zip

In [0]:
! wget http://fenggao-image.stor.sinaapp.com/dogscats.zip
! unzip dogscats.zip

## 2. 数据处理

datasets 是 torchvision 中的一个包，可以用做加载图像数据。它可以以多线程（multi-thread）的形式从硬盘中读取数据，使用 mini-batch 的形式，在网络训练中向 GPU 输送。在使用CNN处理图像时，需要进行预处理。图片将被整理成 $224\times 224 \times 3$ 的大小，同时还将进行归一化处理。

torchvision 支持对输入数据进行一些复杂的预处理/变换 （normalization, cropping, flipping, jittering 等）。具体可以参照 torchvision.tranforms 的官方文档说明。

In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

vgg_format = transforms.Compose([
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])

data_dir = './dogscats'

dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), vgg_format)
         for x in ['train', 'valid']}

dset_sizes = {x: len(dsets[x]) for x in ['train', 'valid']}
dset_classes = dsets['train'].classes

In [0]:
# 通过下面代码可以查看 dsets 的一些属性

print(dsets['train'].classes)
print(dsets['train'].class_to_idx)
print(dsets['train'].imgs[:5])
print('dset_sizes: ', dset_sizes)

In [0]:
loader_train = torch.utils.data.DataLoader(dsets['train'], batch_size=64, shuffle=True, num_workers=6)
loader_valid = torch.utils.data.DataLoader(dsets['valid'], batch_size=5, shuffle=False, num_workers=6)


'''
valid 数据一共有2000张图，每个batch是5张，因此，下面进行遍历一共会输出到 400
同时，把第一个 batch 保存到 inputs_try, labels_try，分别查看
'''
count = 1
for data in loader_valid:
    print(count, end='\n')
    if count == 1:
        inputs_try,labels_try = data
    count +=1

print(labels_try)
print(inputs_try.shape)

In [0]:
# 显示图片的小程序

def imshow(inp, title=None):
#   Imshow for Tensor.
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = np.clip(std * inp + mean, 0,1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [0]:
# 显示 labels_try 的5张图片，即valid里第一个batch的5张图片
out = torchvision.utils.make_grid(inputs_try)
imshow(out, title=[dset_classes[x] for x in labels_try])

## 3. 创建 VGG Model

torchvision中集成了很多在 ImageNet （120万张训练数据） 上预训练好的通用的CNN模型，可以直接下载使用。

在本课程中，我们直接使用预训练好的 VGG 模型。同时，为了展示 VGG 模型对本数据的预测结果，还下载了 ImageNet 1000 个类的 JSON 文件。

在这部分代码中，对输入的5个图片利用VGG模型进行预测，同时，使用softmax对结果进行处理，随后展示了识别结果。可以看到，识别结果是比较非常准确的。

In [0]:
!wget https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json

In [0]:
model_vgg = models.vgg16(pretrained=True)

with open('./imagenet_class_index.json') as f:
    class_dict = json.load(f)
dic_imagenet = [class_dict[str(i)][1] for i in range(len(class_dict))]

inputs_try , labels_try = inputs_try.to(device), labels_try.to(device)
model_vgg = model_vgg.to(device)

outputs_try = model_vgg(inputs_try)

print(outputs_try)
print(outputs_try.shape)

'''
可以看到结果为5行，1000列的数据，每一列代表对每一种目标识别的结果。
但是我也可以观察到，结果非常奇葩，有负数，有正数，
为了将VGG网络输出的结果转化为对每一类的预测概率，我们把结果输入到 Softmax 函数
'''
m_softm = nn.Softmax(dim=1)
probs = m_softm(outputs_try)
vals_try,pred_try = torch.max(probs,dim=1)

print( 'prob sum: ', torch.sum(probs,1))
print( 'vals_try: ', vals_try)
print( 'pred_try: ', pred_try)

print([dic_imagenet[i] for i in pred_try.data])
imshow(torchvision.utils.make_grid(inputs_try.data.cpu()), 
       title=[dset_classes[x] for x in labels_try.data.cpu()])


## 4. 修改最后一层，冻结前面层的参数

VGG 模型如下图所示，注意该网络由三种元素组成：

- 卷积层（CONV）是发现图像中局部的 pattern
- 全连接层（FC）是在全局上建立特征的关联
- 池化（Pool）是给图像降维以提高特征的 invariance

![VGG](http://fenggao-image.stor.sinaapp.com/20191006215625.jpg)

我们的目标是使用预训练好的模型，因此，需要把最后的 nn.Linear 层由1000类，替换为2类。为了在训练中冻结前面层的参数，需要设置 required_grad=False。这样，反向传播训练梯度时，前面层的权重就不会自动更新了。训练中，只会更新最后一层的参数。

In [0]:
print(model_vgg)

model_vgg_new = model_vgg;

for param in model_vgg_new.parameters():
    param.requires_grad = False
model_vgg_new.classifier._modules['6'] = nn.Linear(4096, 2)
model_vgg_new.classifier._modules['7'] = torch.nn.LogSoftmax(dim = 1)

model_vgg_new = model_vgg_new.to(device)

print(model_vgg_new.classifier)

## 5. 训练并测试全连接层

包括三个步骤：第1步，创建损失函数和优化器；第2步，训练模型；第3步，测试模型。

In [0]:
'''
第一步：创建损失函数和优化器

损失函数 NLLLoss() 的 输入 是一个对数概率向量和一个目标标签. 
它不会为我们计算对数概率，适合最后一层是log_softmax()的网络. 
'''
criterion = nn.NLLLoss()

# 学习率
lr = 0.001

# 随机梯度下降
optimizer_vgg = torch.optim.SGD(model_vgg_new.classifier[6].parameters(),lr = lr)

'''
第二步：训练模型
'''

def train_model(model,dataloader,size,epochs=1,optimizer=None):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        running_corrects = 0
        count = 0
        for inputs,classes in dataloader:
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs)
            loss = criterion(outputs,classes)           
            optimizer = optimizer
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
            count += len(inputs)
            print('Training: No. ', count, ' process ... total: ', size)
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.data.item() / size
        print('Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch_loss, epoch_acc))
        
        
# 模型训练
train_model(model_vgg_new,loader_train,size=dset_sizes['train'], epochs=1, 
            optimizer=optimizer_vgg)  

In [0]:
def test_model(model,dataloader,size):
    model.eval()
    predictions = np.zeros(size)
    all_classes = np.zeros(size)
    all_proba = np.zeros((size,2))
    i = 0
    running_loss = 0.0
    running_corrects = 0
    for inputs,classes in dataloader:
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        loss = criterion(outputs,classes)           
        _,preds = torch.max(outputs.data,1)
        # statistics
        running_loss += loss.data.item()
        running_corrects += torch.sum(preds == classes.data)
        predictions[i:i+len(classes)] = preds.to('cpu').numpy()
        all_classes[i:i+len(classes)] = classes.to('cpu').numpy()
        all_proba[i:i+len(classes),:] = outputs.data.to('cpu').numpy()
        i += len(classes)
        print('Testing: No. ', i, ' process ... total: ', size)        
    epoch_loss = running_loss / size
    epoch_acc = running_corrects.data.item() / size
    print('Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch_loss, epoch_acc))
    return predictions, all_proba, all_classes
  
predictions, all_proba, all_classes = test_model(model_vgg_new,loader_valid,size=dset_sizes['valid'])
  

## 6. 可视化模型预测结果（主观分析）

主观分析就是把预测的结果和相对应的测试图像输出出来看看，一般有四种方式：
- 随机查看一些预测正确的图片
- 随机查看一些预测错误的图片
- 预测正确，同时具有较大的probability的图片
- 预测错误，同时具有较大的probability的图片
- 最不确定的图片，比如说预测概率接近0.5的图片

In [0]:
# 单次可视化显示的图片个数
n_view = 8
correct = np.where(predictions==all_classes)[0]
from numpy.random import random, permutation
idx = permutation(correct)[:n_view]
print('random correct idx: ', idx)
loader_correct = torch.utils.data.DataLoader([dsets['valid'][x] for x in idx],
                  batch_size = n_view,shuffle=True)
for data in loader_correct:
    inputs_cor,labels_cor = data
# Make a grid from batch
out = torchvision.utils.make_grid(inputs_cor)
imshow(out, title=[l.item() for l in labels_cor])

# 类似的思路，可以显示错误分类的图片，这里不再重复代码

## 7. 结论

不知道大家发现没有，我们其实只是做了一个简单的 logistic regression！因此，我们实际上相当于是**杀鸡用了牛刀**（kill a fly with a sledge hammer）

在我们这个代码示例中，sledge hammer 是在 ImageNet 上预训练好的 VGG 模型，在这个数据集中，有大量猫和狗的图片。同时，我们也发现，即使不修改网络，模型也可以非常准确的识别猫和狗。

我们学习了冻结前面层，只训练最后的一个 linear layer 中的 8194 个参数 （bias $2\times 4096+2$）。这么一个简单的任务，使用 CPU 训练也是完全可以的。

这个代码示例是非常有启发意义的（instructive），这个实验相当 instructive ，因为它展示的是如何在工程问题中使用深度学习：首先准备待解决问题的数据，然后下载预训练好的网络，接着用准备好的数据来 fine-tune 预训练好的网络。这些步骤在任何深度学习工程项目中都是如此。

最后，期待大家  have fun with the network and understanding the learning process!